In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [16]:
driver = webdriver.Chrome()
url_accidentes = "https://www.one.gob.do/datos-y-estadisticas/"
driver.get(url_accidentes)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/ul/li[2]/a').click()
except:
    print('No encuentro el botón')

sleep(2)

for i in range(2):
    try:
        driver.execute_script("window.scrollBy(0, 500);")  # Desplazarse 1000 píxeles hacia abajo
    except:
        print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/header/div[2]').click()
except:
    print('No encuentro el botón para ampliar')
sleep(2)
try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/header/div[1]').click()
except:
    print('No encuentro el botón para abrir pestañas')
sleep(2)
try:
    driver.execute_script("window.scrollBy(0, 700);")
except:
    print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[1]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo el libro de códigos de accidentes de tráfico

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[8]/td/a').click()
except:
    print('No encuentro el botón para descargar') #descargo los accidentes de tráfico

    

In [2]:
driver = webdriver.Chrome()
url_accidentes = "file:///C:/Users/DELL/Downloads/Libro%20de%20c%C3%B3digos%20de%20defunciones%20en%20accidentes%20de%20tr%C3%A1nsito%202007_2023.htm"
driver.get(url_accidentes)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll
#descargo el excel completo
